In [1]:
import numpy as np

In [130]:
class SequenceGenerator:
    def __init__(self, x, sequence_size, batch_size):
        if x.shape[0] < sequence_size:
            raise ValueError("sequence_size can't be bigger than total number of samples")
        
        if batch_size < 1:
            raise ValueError("batch_size can't be less than 1")
            
        if batch_size > x.shape[0]:
            raise ValueError("batch_size can't be bigger than total number of samples")
            
        self.sequence_size = sequence_size
        self.batch_size = batch_size
        self.x = x
        self.samples_generated = 0
        self.epochs = 0
        self.samples_gen = self._samples_gen()
            
    def _samples_gen(self):
        while True:
            indices = np.random.choice(range(self.x.shape[0] - self.sequence_size - 1), self.batch_size)
            sample_indices = [range(i, i + self.sequence_size) for i in indices]
            targets_indices = indices + self.sequence_size
            batch = np.take(self.x, sample_indices)
            target = np.take(self.x, targets_indices)

            assert batch.shape == (self.batch_size, self.sequence_size)
            assert target.shape == (self.batch_size, )

            yield batch, target
        
    def next_batch(self):
        batch, target = next(self.samples_gen)
        self.samples_generated += target.shape[0]
        if self.samples_generated >= self.x.shape[0]:
            self.epochs += 1
            self.samples_generated = self.x.shape[0] - self.samples_generated
        return batch, target
    
    
class XavierInitializer:
    def do_init(self, in_num, out_num, tensor_shape):
        low_bound = -np.sqrt(6.0 / (in_num + out_num))
        high_bound = np.sqrt(6.0 / (in_num + out_num))
        return np.random.uniform(low_bound, high_bound, size=tensor_shape)

In [133]:
class RNN:
    def __init__(self, units, sequence_size):
        self.units = units
        self.activation = 'tanh'
        self.initializer = XavierInitializer()
        self.sequence_size = sequence_size
        
    def _initialize_tensors(self):
        w_rec_shape = (self.units, self.units)
        w_in_shape = (self.units, self.sequence_size)
        w_out_shape = (self.sequence_size, self.units)
        b_1_shape = (self.units, )
        b_2_shape = (self.sequence_size, )
        
        self.w_rec = self.initializer.do_init(
            self.units * self.units, 
            self.units, 
            w_rec_shape)
        self.w_in = self.initializer.do_init(
            self.units * self.sequence_size, 
            self.units, 
            w_in_shape)
        self.w_out = self.initializer.do_init(
            self.sequence_size * self.units, 
            self.sequence_size, 
            w_out_shape)
        self.b_1 = np.zeros(b_1_shape)
        self.b_2 = np.zeros(b_2_shape)
        
    def fit(self, x, batch_size=32, epochs=32):
        self.batch_generator = SequenceGenerator(x, self.sequence_size, batch_size)
        self._initialize_tensors()
        # while self.batch_generator.epochs < epochs:
            # Forward prop
            
            # Backward prop
        
            # Weight update